# Enunciado

Se diseñará un intercambiador de calor para calentar agua mediante el uso de condensado a 67°C y 0,2 bar, que fluirá en el lado de la carcasa con un caudal másico de 50000 kg/h. Ei calor se transferirá a 30000 kg/h de agua procedente de acueducto a 17°C. Se recomienda una sola carcasa y un solo paso por tubo. Se sugiere una resistencia al ensuciamiento de 0,000176 m2
• K/W y el sobre-diseño de superficie no debe ser superior al 35%.
Se requiere una longitud máxima de tubo de 5 m debido a limitaciones de espacio. El material del tubo es acero al carbono (k = 60 W/m • K). El agua fluirá dentro de tubos 3/4 in (19 mm de diámetro exterior con 16 mm ID). La caída de presión máxima permitida en la carcasa es de 5,0 psi. La temperatura de salida del agua no debe ser inferior a 40°C.

In [82]:
caudal_masico_agua_condensada         = 50_000 / 3600 # kg/s
temperatura_entrada_agua_condensada   = 67 # ºC
presion_agua_condensada               = 0.2 # bar

caudal_transferido                    = 30_000 / 3600 # kg/s
temperatura_entrada                   = 17 # ºC
factor_ensuciamiento                  = 0.000176 # m^2K/W

longitud_maxima                       = 5 # m
k_material                            = 60 # W/(m * K)
diametro_tubos                        = 3/4 # in
diametro_interior                     = 16e-3 # m
diametro_exterior                     = 19e-3 # m

temperatura_salida                    = 40 #ºC

$$ \dot{Q} = (\dot{m} *C_{p})_{c} * (T_{c2} -T_{c1})$$

In [83]:
from pyfluids import Fluid, FluidsList, Input
from core import ThermoFormules
import numpy as np

In [84]:
water = Fluid(FluidsList.Water)
water.name

Water

In [85]:
presion = 101325 * presion_agua_condensada # Pa

In [86]:
temperatura_promedio = (temperatura_entrada + temperatura_salida) /2
temperatura_promedio

28.5

In [87]:
state = water.with_state(
    Input.pressure(presion),
    Input.temperature(temperatura_promedio),
)
state.as_dict()

{'compressibility': 0.00014614052125805355,
 'conductivity': 0.6120463454734684,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 996.0564055844422,
 'dynamic_viscosity': 0.0008233612267610441,
 'enthalpy': 119478.276557885,
 'entropy': 416.01479691577845,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 119457.93132466005,
 'kinematic_viscosity': 8.266210850558526e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 5.623700026792162,
 'pressure': 20265.0,
 'quality': None,
 'sound_speed': 1505.4719747453637,
 'specific_heat': 4180.382725790069,
 'specific_volume': 0.0010039592079258241,
 'surface_tension': None,
 'temperature': 28.5,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [88]:
cp = state.specific_heat
cp

4180.382725790069

In [89]:
calor = caudal_transferido * cp * (temperatura_salida - temperatura_entrada)
calor

801240.0224430967

$$ T_{c2} = - \frac{Q}{(\dot{m} *C_{p})_{c}} + T_{c1}$$

In [90]:
state_cond = water.with_state(
    Input.pressure(presion),
    Input.temperature(40),
)
state_cond.as_dict()

{'compressibility': 0.00014132360145080458,
 'conductivity': 0.6284426249627434,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 992.1807698912999,
 'dynamic_viscosity': 0.000652718564006271,
 'enthalpy': 167544.45024584336,
 'entropy': 572.3967328979538,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 167524.02554033775,
 'kinematic_viscosity': 6.578625426068082e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 4.341067796088867,
 'pressure': 20265.0,
 'quality': None,
 'sound_speed': 1528.762927297536,
 'specific_heat': 4179.614601690888,
 'specific_volume': 0.001007880852306336,
 'surface_tension': None,
 'temperature': 40.0,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [91]:
cp_condensado = state_cond.specific_heat
cp_condensado

4179.614601690888

In [92]:
temperatura_salida_agua_condensada = temperatura_entrada_agua_condensada - calor/(caudal_masico_agua_condensada* cp_condensado)
temperatura_salida_agua_condensada

53.19746385406817

Asuma los coeficiente de trasferencia de calor

$$ \frac{1}{U_f} = \frac{1}{h_{0}} + \frac{r_{0}}{r_{i}} \frac{1}{h_{i}} + R_{ft} + r_{0} \frac{ln(\frac{r_{0}}{r_{i}} )}{k} $$

In [93]:
h_0 = 5_000 # W/m2k
h_1 = 4_000 # W/m2k

In [94]:
U_f = (
        1/h_0 + \
        diametro_exterior/diametro_interior*1/h_1 + \
        factor_ensuciamiento + \
        diametro_exterior/2 * np.log(diametro_exterior/diametro_interior)/k_material
    ) ** -1
U_f

1428.3987473789696

In [95]:
U_cond = (
    1/h_0 + \
    diametro_exterior/diametro_interior*1/h_1 + \
    diametro_exterior/2 * np.log(diametro_exterior/diametro_interior)/k_material
) ** -1
U_cond

1908.0887974581278

In [96]:
diferencia_temperatura_1 = temperatura_entrada_agua_condensada - temperatura_salida
diferencia_temperatura_2 = temperatura_salida_agua_condensada - temperatura_entrada
print(diferencia_temperatura_1, diferencia_temperatura_2)

diferencia_log_temperatura = (diferencia_temperatura_2 - diferencia_temperatura_1)/ np.log(diferencia_temperatura_2/diferencia_temperatura_1)
print(diferencia_log_temperatura)

27 36.19746385406817
31.374365034654137


Asume the F=0.9

In [97]:
F = 0.9
diferencia_log_temperatura_corregida =  F * diferencia_log_temperatura

Estimar las areas requeridas

In [98]:
A_f = calor / (U_f * diferencia_log_temperatura_corregida)
A_c = calor / (U_cond * diferencia_log_temperatura_corregida)

print(A_f, A_c)

19.865327039269854 14.871219985656523


In [101]:
area_ratio =  A_f/A_c - 1
area_ratio

0.3358236283526308

In [103]:
Cl         = 1
L          = 3    # m
CTP        = 0.93 # One tube pass
PitchRatio = 1.25

In [109]:
diametro_carcasa = 0.637 * (Cl/CTP * A_f*pow(PitchRatio,2)* diametro_exterior/L)**(1/2) 
print(diametro_carcasa)
diametro_carcasa = 0.3
print(diametro_carcasa)

0.29286805553111817
0.3


In [112]:
numeros_tubos = 0.785 * (CTP/Cl) * (diametro_carcasa)**2 /((PitchRatio * diametro_exterior)**2)
print(numeros_tubos)

116.48443213296399


Espacio entre deflectors entre 0.4 y 0.6 del diámetro de la carcasa

In [113]:
B = 0.6 * diametro_carcasa
B

0.18